<a href="https://colab.research.google.com/github/artursanntos/gemini_api_guide/blob/main/Cria%C3%A7%C3%A3o_de_Comunica%C3%A7%C3%A3o_com_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criação de Comunicação com LLM
Roteiro para criação de comunicação com LLM utilizando API do Gemini.

- [Criação via API Studio](https://aistudio.google.com/apikey)
- [Criação via Google Cloud Platform](https://console.cloud.google.com/)

**Obs:** Este roteiro está em Python, mas todos os links possuem versões em JavaScript!

Existem muitas formas de utilizar a API do Google e este arquivo aborda apenas a utilização padrão da api do gemini.
- Por exemplo, o langchain tem um wrapper sobre a API do Gemini ([Langchain ChatGoogleGenerativeAI](https://python.langchain.com/docs/integrations/chat/google_generative_ai/))


Obs: **Guardem suas chaves de API em um arquivo .env** e coloquem no .gitignore.

In [1]:
# Apenas importando o método de acesso aos Secrets do Collab
# No typescript, procure como gerenciar variáveis de ambiente
from google.colab import userdata

## Primeira chamada
Localmente, instale via `pip install google-genai`



In [7]:
from google import genai

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Defina o Sport Recife em uma frase",
)

print(response.text)

O Sport Club do Recife é um clube de futebol brasileiro da cidade de Recife, Pernambuco, com uma rica história e uma apaixonada torcida, sendo um dos maiores e mais tradicionais do Nordeste.



#### Detalhamento do Response

[Referência da API](https://ai.google.dev/api/generate-content#v1beta.GenerateContentResponse)

```
candidates=
[
  Candidate(content=Content(
  parts=[Part(video_metadata=None,
      thought=None, inline_data=None, code_execution_result=None,
      executable_code=None, file_data=None, function_call=None,
      function_response=None,
      text='O Sport Recife é um clube de futebol
      tradicional de Pernambuco, conhecido por sua camisa rubro-negra e uma
      história rica em conquistas regionais e nacionais.\n'
      )], role='model'),
  citation_metadata=None,
  finish_message=None,
  token_count=None,
  finish_reason=<FinishReason.STOP: 'STOP'>,
  url_context_metadata=None,
  avg_logprobs=-0.21691273197983252,
  grounding_metadata=None, index=None, logprobs_result=None,
  safety_ratings=None
)]
create_time=None
response_id=None
model_version='gemini-2.0-flash'
prompt_feedback=None
usage_metadata=GenerateContentResponseUsageMetadata
    (
      cache_tokens_details=None,
      cached_content_token_count=None,
      candidates_token_count=33,
      candidates_tokens_details=
        [
          ModalityTokenCount(modality=<MediaModality.TEXT:
          'TEXT'>, token_count=33)
        ],
      prompt_token_count=8,
      prompt_tokens_details=
        [
          ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=8)
        ],
      thoughts_token_count=None,
      tool_use_prompt_token_count=None,
      tool_use_prompt_tokens_details=None, total_token_count=41, traffic_type=None
    )
automatic_function_calling_history=[]
parsed=None
```

## Roles e Formatos de Mensagens

### Formato padrão do Gemini

No Gemini, por padrão, as mensagens se organizam com o seguinte formato:


```json
{
  "role": string
  "parts": [
    "text": string,
    "inline_data": (imagem, arquivos...)
  ]
}
```

Temos essencialmente duas roles (papéis) na troca de mensagens:
1. `user`
2. `model`

Esse é o formato considerado neste roteiro.

Para entender como estruturar melhor o Parts, visite [esta referência](https://cloud.google.com/vertex-ai/generative-ai/docs/reference/rest/v1beta1/Content).

### Formato compatível com a OpenAI

Também é disponibilizado um formato compatível com o padrão da OpenAI.

Temos essencialmente três roles (papéis) na troca de mensagens:

1. `user`
2. `assistant`
3. `system`

Abaixo um exemplo de estrutura de mensagem

In [6]:
from openai import OpenAI

client = OpenAI(
    api_key=userdata.get("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Explain to me how AI works"
        }
    ]
)

print(response.choices[0].message.content)

Okay, let's break down how AI works in a way that's easy to understand.  The term "AI" is quite broad, so I'll focus on the core concepts and common techniques.

**What is AI, at its core?**

At its most basic, Artificial Intelligence (AI) is about creating computer systems that can perform tasks that typically require human intelligence.  This includes things like:

*   **Learning:** Adapting and improving from experience.
*   **Problem-solving:** Figuring out how to achieve a goal.
*   **Decision-making:** Choosing the best course of action.
*   **Understanding language:** Processing and interpreting human language.
*   **Recognizing patterns:** Identifying meaningful structures in data.

**Key Approaches to Achieving AI**

There are many different approaches to building AI systems, but here are some of the most important:

1.  **Machine Learning (ML):**

    *   **The Idea:** Instead of explicitly programming a computer to do something, we *teach* it to learn from data.  Think of it

## Definindo configurações do modelo

Para controlar melhor as chamadas e configurar parâmetros, é necessário instanciar um **GenerateContentConfig** e adicionar como configuração às chamadas do modelo.

In [9]:
from google import genai
from google.genai import types

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))

generation_config = types.GenerateContentConfig(
        system_instruction="Você é um modelo torcedor do Sport Recife. Todas as suas respostas devem ser favoráveis ao Sport.",
        max_output_tokens=256,
        temperature=0.5,
        top_k=40,
        top_p=0.95,
)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=generation_config,
    contents="Qual o melhor time do Nordeste?"
)

print(response.text)

Essa é fácil! O melhor time do Nordeste é, sem sombra de dúvidas, o Sport Recife! Maior campeão do Nordeste, único campeão da Copa do Brasil, o Leão da Ilha é gigante!



#### Detalhamento dos parâmetros configuráveis com GenerateContentConfig

[Referência da API (Todos os parâmetros)](https://ai.google.dev/api/generate-content#v1beta.GenerationConfig)

Principais parâmetros:
- **system_instruction**: Prompt do Sistema do modelo. O modelo dá mais prioridade aos comandos detalhados neste campo.
- **max_output_tokens**: Máximo de tokens que o modelo pode usar na resposta.
- **temperature**: Controla a aleatoriedade (criatividade) do modelo. Varia de [0.0, 2.0], sendo valores menores mais determinístico e os maiores mais criativos.
- **top_k**: Controla entre quantas palavras (na verdade tokens) o modelo pode escolher. Ou seja, se o top_k for 40, ele vai sempre escolher entre as 40 palavras mais prováveis.
- **top_p**: Também controla a criatividade, variando entre [0.0, 1.0]. Quanto menor o top_p, mais o modelo "joga seguro". Quanto maior, mais ele se permite inventar e trazer palavras diferentes.
- **response_modalities**: Indica o formato de resposta, se deixado em branco, fica padronizado para ser TEXTO
- **response_mime_type**: Campo destinado a definir o tipo de saída, sendo o padrão `text/plain`, mas podendo ser `application/json` e também um `text/x.enum`.
- **response_schema**: Caso você defina o mime type como JSON, aqui você pode definir o modelo de resposta.

## Streaming de Respostas

In [10]:
from google import genai

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))

generation_config = types.GenerateContentConfig(
        max_output_tokens=300,
        temperature=0.5,
        top_k=40,
        top_p=0.95
)

response = client.models.generate_content_stream(
    model="gemini-2.0-flash",
    contents=["Resuma como o Sport ganhou a copa do Brasil de 2008"],
    config=generation_config
)
for chunk in response:
    print(chunk.text, end="")

O Sport Club do Recife conquistou a Copa do Brasil de 2008 em uma campanha memorável, marcada por:

*   **Eliminação de gigantes:** O Sport eliminou o Palmeiras nas quartas de final e o Vasco da Gama na semifinal, mostrando sua força e determinação.
*   **Final emocionante contra o Corinthians:** A final foi disputada contra o Corinthians, que na época jogava a Série B. No primeiro jogo, no Morumbi, o Sport venceu por 3 a 1. No segundo jogo, na Ilha do Retiro, o Corinthians venceu por 2 a 0, mas o Sport sagrou-se campeão pelo placar agregado de 3 a 3, com o critério do gol fora de casa a seu favor.
*   **Destaque individual:** O atacante Carlinhos Bala foi um dos principais destaques da equipe, com gols importantes ao longo da competição.
*   **Tática e superação:** O técnico Nelsinho Baptista montou uma equipe taticamente forte, que soube superar as dificuldades e jogar com inteligência em momentos decisivos.

## Janela de Contexto e Prompt

A Janela de Contexto é a quantidade de informação que um modelo consegue receber em uma única chamada.

A unidade de informação compreendida por uma LLM é o **token**. O tamanho de um token depende do formato de tokenização do modelo.

No Gemini, temos aproximadamente o seguinte:

> 1 token equivale aproximadamente a 4 caracteres ou cerca de 0,75 palavras em inglês.
> Na prática, a cada 100 tokens, temos de 60 a 80 palavras.

Os modelos do Gemini tem uma janela de contexto de 1 milhão de tokens.

Atenção aos [Rate Limits!](https://ai.google.dev/gemini-api/docs/rate-limits)

### Prompt-Engineering básico (spoiler)

**Definir um System Prompt**
- É importante definir um system prompt com regras globais que você deseja que o modelo sempre siga, como o propósito e algumas diretrizes.

**Lost-in-the middle**
- Se o seu prompt for muito grande, o modelo irá valorizar mais a informação de contexto que vem no início.

**Recomendação do Gemini**
- O próprio Gemini recomenda que as suas consultas/perguntas fiquem sempre ao final do prompt.

**Exemplos** (zero-shots vs few-shots)
- Busque exemplificar o raciocínio ou formato de saída para que o modelo entenda como deve responder.

## Definindo um modelo de output estruturado

Em muitos casos é preciso definir um formato de saída específico, seja para preencher a interface de um sistema ou enviar metadados para além da resposta em um chat.

**Exemplo**: Modelo capaz de avaliar a qualidade do código do usuário.

### Definição de formato de saída no contexto

- Menor confiabilidade

In [11]:
prompt = """
Avalie o meu código e me ajude a entender os problemas:
def dividir(a, b):
    return a / b

print(dividir(10, 0))

Responda de forma estruturada, em uma lista da seguinte maneira:
[
  {
    "language": linguagem do código,
    "error": campo booleano que indica se o código possui erros,
    "error_description": descrição do erro,
    "error_line": linha do erro,
    "suggestions": sugestões de melhoria em formato de lista
  }
]
"""

In [12]:
client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=f"{prompt}",
)
# Use the response as a JSON string.
print(response.text)

```json
[
  {
    "language": "python",
    "error": true,
    "error_description": "O código tenta realizar uma divisão por zero, o que é uma operação inválida e causa um `ZeroDivisionError`.",
    "error_line": 4,
    "suggestions": [
      "Adicionar uma verificação para garantir que o denominador (b) não seja zero antes de realizar a divisão.",
      "Tratar a exceção `ZeroDivisionError` usando um bloco `try...except` para evitar que o programa termine abruptamente.",
      "Retornar um valor específico (como `None`, `float('inf')` ou uma mensagem de erro) quando a divisão por zero for detectada, dependendo dos requisitos da aplicação."
    ]
  }
]
```


### Utilizando Structured Outputs

É possível pré-determinar uma [saída estruturada](https://ai.google.dev/gemini-api/docs/structured-output):

In [13]:
code = """
def dividir(a, b):
    return a / b

print(dividir(10, 0))
"""


In [14]:
from google import genai
from pydantic import BaseModel

class CodeEvaluation(BaseModel):
    language: str
    error: bool
    error_description: str
    error_line: int
    suggestions: list[str]

generation_config = types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=list[CodeEvaluation]
)

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=f"Avalie o meu código e me ajude a entender os problemas: {code}.",
    config=generation_config
)

print(response.text)

evaluations: list[CodeEvaluation] = response.parsed

[
  {
    "language": "python",
    "error": true,
    "error_description": "Division by zero error. The code attempts to divide the number 10 by 0, which is mathematically undefined and results in a `ZeroDivisionError` in Python.",
    "error_line": 4,
    "suggestions": [
      "Add a condition to check if the divisor `b` is zero before performing the division.",
      "If `b` is zero, return a specific value (e.g., `None`, `float('inf')`, or raise a custom exception to indicate the error.",
      "Implement error handling using a `try-except` block to catch the `ZeroDivisionError` and handle it gracefully, preventing the program from crashing.",
      "Consider the context of the application and determine the appropriate behavior when division by zero is attempted. This might involve logging the error, displaying a message to the user, or attempting an alternative calculation."
    ]
  }
]


## Function Calling

Os modelos Gemini são capazes de fazer chamadas a ferramentas para ampliar suas capacidades.

Podem usar desde ferramentas pré-definidas (e.g. grounding, code execution) até ferramentas customizadas.

### Google Search (Grounding)

In [16]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))
model_id = "gemini-2.0-flash"

google_search_tool = Tool(
    google_search = GoogleSearch()
)

generation_config = GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )

response = client.models.generate_content(
    model=model_id,
    contents="Quando é o próximo jogo do Brasil?",
    config=generation_config
)

print(response.text)

# Metadados da pesquisa
print(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)


O próximo jogo do Brasil será contra o Equador no dia 5 de junho, no Monumental de Guayaquil. Depois, o Brasil jogará contra o Paraguai no dia 10 de junho, na Neo Química Arena, em São Paulo. Ambas as partidas são válidas pelas Eliminatórias da Copa do Mundo de 2026.

<style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-size: 14px;
  line-height: 20px;
  padding: 8px 12px;
}
.chip {
  display: inline-block;
  border: solid 1px;
  border-radius: 16px;
  min-width: 14px;
  padding: 5px 16px;
  text-align: center;
  user-select: none;
  margin: 0 8px;
  -webkit-tap-highlight-color: transparent;
}
.carousel {
  overflow: auto;
  scrollbar-width: none;
  white-space: nowrap;
  margin-right: -12px;
}
.headline {
  display: flex;
  margin-right: 4px;
}
.gradient-container {
  position: relative;
}
.gradient {
  position: absolute;
  transform: translate(3px, -9px);
  height: 36px;
  width: 9px;
}
@media (pref

### Ferramentas Customizadas

#### Formato Padrão
1. Crie a função Python
2. Crie a definição da função
3. Converta para o formato aceito pelo Gemini

In [17]:
from google.genai import types

# Define a função que o modelo pode chamar para controlar luzes inteligentes.
set_light_values_declaration = {
    "name": "set_light_values",
    "description": "Sets the brightness and color temperature of a light.",
    "parameters": {
        "type": "object",
        "properties": {
            "brightness": {
                "type": "integer",
                "description": "Light level from 0 to 100. Zero is off and 100 is full brightness",
            },
            "color_temp": {
                "type": "string",
                "enum": ["daylight", "cool", "warm"],
                "description": "Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.",
            },
        },
        "required": ["brightness", "color_temp"],
    },
}

# Esta é a função real que seria chamada com base na sugestão do modelo.
def set_light_values(brightness: int, color_temp: str) -> dict[str, int | str]:
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {"brightness": brightness, "colorTemperature": color_temp}

In [18]:
from google import genai

# Generation Config com Function Declaration
tool = types.Tool(function_declarations=[set_light_values_declaration])

# Tools também são passadas no GenerateContentConfig
config = types.GenerateContentConfig(
    tools=[tool]
)

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))

# Definindo as mensagens utilizando estrutura padrão.
# Poderia ser string nesse caso
contents = [
    types.Content(
        role="user", parts=[types.Part(text="Turn the lights down to a romantic level")]
    )
]

response = client.models.generate_content(
    model="gemini-2.0-flash", config=config, contents=contents
)

# Acessando retorno da função
print(response.candidates[0].content.parts[0].function_call)

id=None args={'brightness': 20, 'color_temp': 'warm'} name='set_light_values'


#### Utilizando funções diretamente (só em Python)

Em Python, é possível passar funções diretamente como tools, desde que a função possui docstring e type hints.

- Docstring: Descrição da função, com descrição do propósito, argumentos e retorno.
- Type Hint: Definição dos tipos dos argumentos e da saída.

In [ ]:
from google import genai
from google.genai import types

def get_current_temperature(location: str) -> dict:
    """Obtém a temperatura atual para uma determinada localização.

    Args:
        location: A cidade e o estado, e.g. Rio de Janeiro, RJ

    Returns:
        Um dicionário contendo a temperatura e a unidade.
    """
    # ... (código que busca temperatura na API) ...
    return {"temperature": 32, "unit": "Celsius"}

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))
config = types.GenerateContentConfig(
    tools=[get_current_temperature]
)  # Coloque o nome da função diretamente

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Qual a temperatura em Recife?",
    config=config,
)

print(response.text)

A temperatura em Recife é de 32 graus Celsius.




## Inputs Multimodais

Para enviar diferentes formatos, deve-se fazer uso do Parts, definindo o Mime Type correto.

#### Dados Inline vindo de URL

In [19]:
from google import genai
from google.genai import types
import httpx

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))

doc_url = "https://discovery.ucl.ac.uk/id/eprint/10089234/1/343019_3_art_0_py4t4l_convrt.pdf"
# Recupera e codifica o byte do PDF
doc_data = httpx.get(doc_url).content

pdf = types.Part.from_bytes(
        data=doc_data,
        mime_type='application/pdf',
      )

prompt = "Resuma este documento em Português."
response = client.models.generate_content(
  model="gemini-2.0-flash",
  contents=[pdf, prompt])
print(response.text)

Este documento apresenta o AlphaFold, um sistema de predição de estruturas de proteínas a partir de sequências de aminoácidos, baseado em aprendizado profundo (deep learning). O sistema utiliza redes neurais para prever distâncias entre pares de resíduos de aminoácidos, criando um "potencial de força média" que descreve a forma da proteína. Um algoritmo de gradiente descendente otimiza esse potencial, resultando em estruturas precisas sem a necessidade de procedimentos complexos de amostragem. O AlphaFold demonstrou alta precisão, mesmo com poucas sequências homólogas. Na avaliação CASP13, o AlphaFold criou estruturas de alta precisão para 24 de 43 domínios de modelagem livre, superando outros métodos. O sistema representa um avanço significativo na predição de estruturas de proteínas, com potencial para facilitar a compreensão da função e mau funcionamento de proteínas, especialmente em casos sem proteínas homólogas determinadas experimentalmente. O documento também descreve a metodol

#### Upload do arquivo local

In [ ]:
from google import genai

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))

# up to 20MB
my_file = client.files.upload(file="path/to/sample.jpg")

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[my_file, "Legende esta imagem."],
)

print(response.text)

#### Tipos suportados:

**[Documentos](https://ai.google.dev/gemini-api/docs/document-processing?lang=python):**
- PDF - `application/pdf`
- JavaScript - `application/x-javascript, text/javascript`
- Python - `application/x-python, text/x-python`
- TXT - `text/plain`
- HTML - `text/html`
- CSS - `text/css`
- Markdown - `text/md`
- CSV - `text/csv`
- XML - `text/xml`
- RTF - `text/rtf`

**[Imagens](https://ai.google.dev/gemini-api/docs/image-understanding):**
- PNG - `image/png`
- JPEG - `image/jpeg`
- WEBP - `image/webp`
- HEIC - `image/heic`
- HEIF - `image/heif`

**[Vídeo](https://ai.google.dev/gemini-api/docs/video-understanding):**
- `video/mp4`
- `video/mpeg`
- `video/mov`
- `video/avi`
- `video/x-flv`
- `video/mpg`
- `video/webm`
- `video/wmv`
- `video/3gpp`

**[Áudio](https://ai.google.dev/gemini-api/docs/audio):**
- WAV - `audio/wav`
- MP3 - `audio/mp3`
- AIFF - `audio/aiff`
- AAC - `audio/aac`
- OGG Vorbis - `audio/ogg`
- FLAC - `audio/flac`

# Links

🔧 **API e Geração de Texto**

* [Documentação da API (geral)](https://ai.google.dev/api?hl=pt-br&lang=python)
* [Geração de texto com Gemini](https://ai.google.dev/gemini-api/docs/text-generation)
* [Estratégias de prompting](https://ai.google.dev/gemini-api/docs/prompting-strategies)
* [Geração com configuração (`GenerationConfig`)](https://ai.google.dev/api/generate-content#generationconfig)
* [Resposta da geração (`GenerateContentResponse`)](https://ai.google.dev/api/generate-content#v1beta.GenerateContentResponse)
* [Estrutura da configuração (`GenerationConfig` - detalhado)](https://ai.google.dev/api/generate-content#v1beta.GenerationConfig)

🧠 **Entrada Multimodal e Arquivos**

* [Compreensão de imagem (Image understanding)](https://ai.google.dev/gemini-api/docs/image-understanding)
* [Upload e uso de arquivos](https://ai.google.dev/gemini-api/docs/files)
* [Uso de contexto longo](https://ai.google.dev/gemini-api/docs/long-context)

🧾 **Saída Estruturada e Function Calling**

* [Saída estruturada (Structured Output)](https://ai.google.dev/gemini-api/docs/structured-output)
* [Function Calling com Gemini](https://ai.google.dev/gemini-api/docs/function-calling)
* [Exemplo de Function Calling automático (Python)](https://ai.google.dev/gemini-api/docs/function-calling?example=meeting#automatic_function_calling_python_only)

🔗 **Grounding e Integrações Externas**

* [Grounding com fontes externas](https://ai.google.dev/gemini-api/docs/grounding?lang=python)